# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [34]:
# prompt: library opencv

!pip install opencv-python


In [35]:
!pip install ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [36]:
from ultralytics import YOLO
# Load a model
model = YOLO("YOLO_Model.pt")  # load a pretrained model (recommended for training)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [37]:
import cv2

In [38]:
VIDEO_PATH='/content/YOLO_Video.mp4'
cap=cv2.VideoCapture(VIDEO_PATH)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [39]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [40]:
output_path = '/content/output_video_with_predictions.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [44]:
# with Process Video Frames and Identify Vehicles Left the Road Form Each Street

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [45]:
box_color = (0, 0, 255)
text_color = (0, 255, 0)
center_color = (0, 0, 255)
thickness = 2

In [46]:
top, bottom, left, right = 0, 0, 0, 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = model(frame)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy()

        for box, conf, class_id in zip(boxes, confidences, class_ids):
            x1, y1, x2, y2 = map(int, box)
            label = f'ID:{int(class_id)} Conf:{conf:.2f}'

            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, thickness)

            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.circle(frame, (center_x, center_y), 5, center_color, -1)

            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, thickness)


            if top_roi[1] <= center_y <= top_roi[3] and top_roi[0] <= center_x <= top_roi[2]:
                top += 1
            elif bottom_roi[1] <= center_y <= bottom_roi[3] and bottom_roi[0] <= center_x <= bottom_roi[2]:
                bottom += 1
            elif left_roi[1] <= center_y <= left_roi[3] and left_roi[0] <= center_x <= left_roi[2]:
                left += 1
            elif right_roi[1] <= center_y <= right_roi[3] and right_roi[0] <= center_x <= right_roi[2]:
                right += 1

    cv2.putText(frame, 'RAZAN ALHARBI', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 3)

    top_roi = (600, 0, 1200, 150)
    bottom_roi = (600, 900, 1200, height)
    left_roi = (0, 300, 300, 700)
    right_roi = (1500, 300, width, 700)

    yellow_color = (0, 255, 255)
    thickness = 2

    cv2.rectangle(frame, (top_roi[0], top_roi[1]), (top_roi[2], top_roi[3]), yellow_color, thickness)
    cv2.rectangle(frame, (bottom_roi[0], bottom_roi[1]), (bottom_roi[2], bottom_roi[3]), yellow_color, thickness)
    cv2.rectangle(frame, (left_roi[0], left_roi[1]), (left_roi[2], left_roi[3]), yellow_color, thickness)
    cv2.rectangle(frame, (right_roi[0], right_roi[1]), (right_roi[2], right_roi[3]), yellow_color, thickness)
    cv2.putText(frame, f'Top Exits: {top}', (50, height - 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    cv2.putText(frame, f'Bottom Exits: {bottom}', (50, height - 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    cv2.putText(frame, f'Left Exits: {left}', (50, height - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    cv2.putText(frame, f'Right Exits: {right}', (50, height - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 45.8ms
Speed: 3.3ms preprocess, 45.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 49.4ms
Speed: 8.6ms preprocess, 49.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 42.6ms
Speed: 3.7ms preprocess, 42.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 42.1ms
Speed: 3.1ms preprocess, 42.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 33.7ms
Speed: 3.1ms preprocess, 33.7ms inferenc

## Save and Submit Your Work

Submit both the notebook and the output video